In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Load data

In [48]:
# read data

data = pd.read_table('./data/ml-100k/u.data', header=None, 
                     names=['userId', 'itemId', 'rating', 'timestamp'])

user = pd.read_table('./data/ml-100k/u.user', header=None, sep='|', 
                     names=['userId', 'age', 'gender', 'occupation', 'zipcode'])

item = pd.read_table('./data/ml-100k/u.item', header=None, sep='|', engine='python',
                     names=['itemId', 'title', 'date', 'video date', 'URL', 'Unknown', 
                            'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 
                            'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                            'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
                            'Thriller', 'War', 'Western'])

In [51]:
data.head()

,userId,itemId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [53]:
user.head()

,userId,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [54]:
item.head()

,itemId,title,date,video date,URL,Unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [57]:
# read five folders for cross-validation
names = ['userId', 'itemId', 'rating', 'timestamp']
u1_base = pd.read_table('./data/ml-100k/u1.base', header=None, names=names)
u2_base = pd.read_table('./data/ml-100k/u2.base', header=None, names=names)
u3_base = pd.read_table('./data/ml-100k/u3.base', header=None, names=names)
u4_base = pd.read_table('./data/ml-100k/u4.base', header=None, names=names)
u5_base = pd.read_table('./data/ml-100k/u5.base', header=None, names=names)

u1_test = pd.read_table('./data/ml-100k/u1.test', header=None, names=names)
u2_test = pd.read_table('./data/ml-100k/u2.test', header=None, names=names)
u3_test = pd.read_table('./data/ml-100k/u3.test', header=None, names=names)
u4_test = pd.read_table('./data/ml-100k/u4.test', header=None, names=names)
u5_test = pd.read_table('./data/ml-100k/u5.test', header=None, names=names)

# Pre-processing

# Part a: Build Model

# Part b: Visualization

# Part c: Re-build Model